<a href="https://colab.research.google.com/github/Saifullah785/deep-learning-projects/blob/main/Project_11_dog_breeds_detection_transfer_learning/Project_11_dog_breeds_detection_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load all Packages

In [ ]:
# !pip install kaggle

In [ ]:
# upload kaggle API token: then proceed
# move api token
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
# download dataset

!kaggle dataset download -d 'mohamedchahed/dog-breeds'

# unzip dataset
!unzip dog-breeds.zip

In [ ]:
import os
import random
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Sequential

from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler

# Data Augmentation

In [ ]:
# creating IMageDataGenerator instance to Augment,split and then pass our images to the model

data_gen = ImageDataGenerator(rescale = 1./255,
                              width_shift_range = 0.2,
                              validation_split= 0.1,
                              height_shift_range= 0.2,
                              shear_range = 0.2,
                              horizontal_flip = True,
                              vertical_flip = True,
                              zoom_range = 0.2)

#Applying Data Augementation On both training and validation sets

In [ ]:
# Creating our test and validation data generator to flow images to network from images directory

training_data = data_gen.flow_from_directory('/content/do-breeds',
                                             target_size = (224,224),
                                             class_mode = 'categorical',
                                             subset = 'training')

validation_data = data_gen.flow_from_directory('/content/do-breeds',
                                               target_size = (224,224),
                                               class_mode = 'categorical',
                                               subset = 'validation')

# Downloading a Pretrained InceptionV3 Model by URL

In [ ]:
# Downloading the pre-trained weights.
# No top means it excludes the fully connected layer it uses for classification. we only need convolution layers.

# In colab or kaggle notebook use the following
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O //content/sample_data/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


# Transfer Learning Intialization and Configuration

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the weights file you download into a variable
local_weights_file = '/content/sample_data/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


# Initialize the base model.
# Set the input shape and remove the dense layers.

pre_trained_model = InceptionV3(input_shape = (224, 224, 3),
                                include_top = False,
                                weights = None)

# Load the pre-trained weights you downloded.
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of th layers
# As said before we don't want to re-train model's convolutional layers
